In [6]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [84]:
#create custom Dataset
encoder = LabelEncoder()
class CustomDataset(Dataset):
    def __init__(self):
        data = pd.read_csv("./dataset/IRIS.csv", sep=",", header=1)
        self.y = encoder.fit_transform(data.iloc[::1,4]) # map 0,1,2 to labels
        self.x = torch.from_numpy(data.iloc[:,:-1].values).float() # classes converted from df to tensor
        self.len = len(data) 
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [78]:
#load dataset and split in training and testset
dataset = CustomDataset()
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [79]:
batch_size = 10
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [80]:
#create NN-Network
#4 inputs and 3  outputs 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack =  nn.Sequential(
         nn.Linear(4,128),
         nn.ReLU(),
         nn.Linear(128,64),
         nn.ReLU(),
         nn.Linear(64,3)
         )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits

In [81]:
lr = 0.01
model = NeuralNetwork()
optim = torch.optim.Adam(model.parameters(), lr=lr)
#optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
loss = torch.nn.CrossEntropyLoss()

next(iter(train_loader))

[tensor([[6.7000, 3.1000, 4.4000, 1.4000],
         [6.5000, 3.0000, 5.5000, 1.8000],
         [4.9000, 3.1000, 1.5000, 0.1000],
         [6.7000, 3.3000, 5.7000, 2.5000],
         [5.5000, 2.4000, 3.8000, 1.1000],
         [6.0000, 2.7000, 5.1000, 1.6000],
         [6.1000, 2.9000, 4.7000, 1.4000],
         [4.9000, 3.0000, 1.4000, 0.2000],
         [5.4000, 3.7000, 1.5000, 0.2000],
         [6.3000, 2.3000, 4.4000, 1.3000]]),
 tensor([1, 2, 0, 2, 1, 1, 1, 0, 0, 1])]

In [83]:
n_iters = 1000
steps = n_iters/10
LOSS = []
for epochs in range(n_iters):  
    for i,(inputs, labels) in enumerate(train_loader):
        optim.zero_grad()
        out = model(inputs)
        l = loss(out, labels)
        l.backward()
        optim.step()
    LOSS.append(l.item())
    if epochs%steps == 0:
        print(f"\n epoch: {int(epochs+steps)}/{n_iters}, loss: {sum(LOSS)/len(LOSS)}")
        


 epoch: 100/1000, loss: 0.7453175187110901

 epoch: 200/1000, loss: 0.11550193479731863

 epoch: 300/1000, loss: 0.10289594598856439

 epoch: 400/1000, loss: 0.09079051319312509

 epoch: 500/1000, loss: 0.08833129146888749

 epoch: 600/1000, loss: 0.08667028534785821

 epoch: 700/1000, loss: 0.08149761736809427

 epoch: 800/1000, loss: 0.09837388034251177

 epoch: 900/1000, loss: 0.09310443310659756

 epoch: 1000/1000, loss: 0.09168043074542413


In [86]:
#Eval model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Y_pred = []
Y_hat = []
with torch.no_grad():
    for i,(inputs, labels) in enumerate(test_loader):
        out = model(inputs.float())
        _, out_hat = torch.max(out,1)
        Y_pred.append(out_hat)
        Y_hat.append(labels)
          
Y_pred = torch.cat(Y_pred)
Y_hat = torch.cat(Y_hat)
Y_hat.cpu().detach().numpy()
Y_pred.cpu().detach().numpy()

#Y_hat,Y_pred
print(classification_report(y_true=Y_hat, y_pred=Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      0.88      0.93         8
           2       0.90      1.00      0.95         9

    accuracy                           0.97        29
   macro avg       0.97      0.96      0.96        29
weighted avg       0.97      0.97      0.97        29



In [89]:
#test manual witch rnd datapoint:
output_test = model(torch.tensor([4.5,2.3,1.3,0.3]))
output_test

tensor([ 74.5165, -42.4948, -58.8890], grad_fn=<AddBackward0>)

In [102]:
df = pd.read_csv("./dataset/IRIS.csv", sep=",")
y = df.iloc[::1,4]
encoder.fit_transform(y)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [108]:
output_test = torch.argmax(output_test)
flower = output_test.item()

tensor(0)

In [111]:
encoder.inverse_transform([flower]).item()

'Iris-setosa'